In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import ast

from fastDamerauLevenshtein import damerauLevenshtein
from opportunistic_planning import data, sequence
from imp import reload
from scipy.stats import friedmanchisquare, wilcoxon
reload(data)
reload(sequence)
pd.set_option('display.max_rows', 500)

In [2]:
# read data

df = pd.read_csv('all_task_environments_2021-01-26.csv', header=0)

In [3]:
# dict generation ~1 sec

%time distances_dict = data.generate_distances_dict(df)

CPU times: user 1.1 s, sys: 64.1 ms, total: 1.16 s
Wall time: 1.05 s


In [29]:
# n=1: ~4min
# n=10: ~15min
# n=50: ~1h 44min
# n=100: 2h 4min
%time results_new, results_sum = data.calculate_edit_distances_prequential(df, distances_dict=distances_dict, n=50)

CPU times: user 1h 44min 2s, sys: 5.93 s, total: 1h 44min 8s
Wall time: 1h 44min 7s


In [12]:
results_sum

,"c: 1.0; k: 0.0,0.1,1.1; x","c: 1.0; k: 0.0,0.1,1.1; y","c: 1.0; k: 0.0,0.1,1.1; z","c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.0; k: 0.0,0.1,1.1; xz","c: 1.0; k: 0.0,0.1,1.1; yz","c: 1.0; k: 0.0,0.1,1.1; xyz","c: 1.1; k: 0.0,0.1,1.1; x","c: 1.1; k: 0.0,0.1,1.1; y","c: 1.1; k: 0.0,0.1,1.1; z",...,"c: 1.9; k: 0.8,0.9,1.9; x","c: 1.9; k: 0.8,0.9,1.9; y","c: 1.9; k: 0.8,0.9,1.9; z","c: 1.9; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xz","c: 1.9; k: 0.8,0.9,1.9; yz","c: 1.9; k: 0.8,0.9,1.9; xyz",sequence,error,ID
0,4.0000,3.000000,4.0000,3.000000,4.000000,3.000000,3.000000,4.000000,3.000000,4.0000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,pocgkr,0.723,a1
1,7.0000,5.000000,4.0000,6.000000,7.000000,5.000000,6.000000,7.000000,5.000000,4.0000,...,7.000000,6.000000,3.000000,7.000000,7.000000,6.000000,7.000000,cgwpcfks,0.785,a3
2,5.0000,7.000000,4.0000,6.000000,5.000000,7.000000,6.000000,5.000000,7.000000,4.0000,...,4.000000,4.000000,7.000000,4.000000,4.000000,4.000000,4.000000,kfsfkspwg,0.806,a5
3,7.0000,9.000000,9.0000,9.000000,7.000000,9.000000,9.000000,7.000000,9.000000,9.0000,...,7.000000,5.000000,9.000000,7.000000,7.000000,5.000000,7.000000,pfkswkfsococg,0.862,a11
4,6.0000,6.000000,6.0000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.0000,...,6.000000,4.000000,3.000000,6.000000,6.000000,4.000000,6.000000,wptgkfsoc,0.806,a13
5,2.0000,2.000000,2.0000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.0000,...,2.000000,3.000000,2.000000,3.000000,2.000000,3.000000,3.000000,pwfkg,0.666,a16
6,3.0000,3.000000,5.0000,4.000000,3.000000,4.000000,4.000000,3.000000,3.000000,5.0000,...,3.000000,4.000000,5.000000,4.000000,3.000000,5.000000,4.000000,powcer,0.723,a19
7,6.0000,4.000000,5.0000,5.000000,6.000000,5.000000,5.000000,6.000000,4.000000,5.0000,...,5.000000,6.000000,4.000000,6.000000,5.000000,6.000000,6.000000,coprerk,0.756,a22
8,7.0000,4.000000,7.0000,5.000000,7.000000,4.000000,5.000000,7.000000,4.000000,7.0000,...,7.000000,6.000000,6.000000,7.000000,7.000000,6.000000,7.000000,wopofkkskf,0.824,a26
9,5.0000,6.000000,6.0000,4.000000,5.000000,4.000000,4.000000,5.000000,6.000000,6.0000,...,4.000000,6.000000,4.000000,4.000000,4.000000,4.000000,4.000000,opwpwgcoc,0.806,a29


In [42]:
#results_new.T.to_csv('results/results_all_n50_2021-01-28.csv', header=True, index=True)

In [30]:
lowest_sum, lowest_idx_sum, lowest_median_sum, list_mean_sum, results_mean_sum = data.get_lowest_error(results_sum)
lowest_sum, lowest_idx_sum, lowest_median_sum

(4.126888888888888, Index(['c: 1.5; k: 0.2,0.3,1.6; xy'], dtype='object'), 4.0)

In [10]:
def get_list_sorted(results):
    
    for col in list(results):
        if col != 'sequence' and col != 'error' and col != 'ID':
            results.loc['mean', col] = results[col].mean()
            results.loc['median', col] = results[col].median()
    lowest = sorted(results.loc['mean'])
    lowest_median = sorted(results.loc['median'])
    
    idx_list = []
                
    for i in lowest_median:
        idx = results.columns[(results.loc['median'] == i)]
        idx_list.append(idx)

    return lowest, results.columns[(results.loc['mean'] == lowest[0])], lowest_median, idx_list

In [41]:
#low, idx, low_median, idx_list = get_list_sorted(results_sum)
[(val, i) for val, i in zip(low_median[:2],idx_list[:2])]

[(4.0,
  Index(['c: 1.0; k: 0.0,0.1,1.1; xy', 'c: 1.0; k: 0.0,0.1,1.1; xyz',
         'c: 1.1; k: 0.0,0.1,1.1; xy', 'c: 1.1; k: 0.0,0.1,1.1; xyz',
         'c: 1.2; k: 0.0,0.1,1.1; xy', 'c: 1.2; k: 0.0,0.1,1.1; xyz',
         'c: 1.3; k: 0.0,0.1,1.1; xy', 'c: 1.3; k: 0.0,0.1,1.1; xyz',
         'c: 1.4; k: 0.0,0.1,1.1; xy', 'c: 1.4; k: 0.0,0.1,1.1; xyz',
         ...
         'c: 1.2; k: 0.6,0.7,1.9; xyz', 'c: 1.3; k: 0.6,0.7,1.9; xy',
         'c: 1.3; k: 0.6,0.7,1.9; xyz', 'c: 1.4; k: 0.6,0.7,1.9; xy',
         'c: 1.5; k: 0.6,0.7,1.9; xy', 'c: 1.6; k: 0.6,0.7,1.9; xy',
         'c: 1.8; k: 0.6,0.7,1.9; xy', 'c: 1.8; k: 0.6,0.7,1.9; xyz',
         'c: 1.9; k: 0.6,0.7,1.9; xy', 'c: 1.9; k: 0.6,0.7,1.9; xyz'],
        dtype='object', length=1173)),
 (4.0,
  Index(['c: 1.0; k: 0.0,0.1,1.1; xy', 'c: 1.0; k: 0.0,0.1,1.1; xyz',
         'c: 1.1; k: 0.0,0.1,1.1; xy', 'c: 1.1; k: 0.0,0.1,1.1; xyz',
         'c: 1.2; k: 0.0,0.1,1.1; xy', 'c: 1.2; k: 0.0,0.1,1.1; xyz',
         'c: 1.3; k: 0.0

### Plot dimensions

In [14]:
# Define input for plots

c = [float(x[3:6]) for x in results_sum.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
k = [float(x[11:14]) for x in results_sum.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
dim = [x.strip() for x in results_sum.columns[:-3].str.split(';').str[2]]
median_dims = [x for x in results_mean_sum.loc['mean'].tolist() if pd.notnull(x)]

In [22]:
%matplotlib qt

#cm = ['red','blue','green','magenta','cyan','orange','grey']
cm = ['#e74c3c', '#0064a3', '#70b85d', '#287d78', '#54d0ff', '#f1e664', '#fd8f00']
dim_num = [0 if x=='x' else 1 if x=='y' else 2 if x=='z' else 3 if x=='xy' else 4 if x=='xz' else 5 if x=='yz' else 6 for x in dim]

cmap = matplotlib.colors.ListedColormap(cm)

ticks = ['x', 'y', 'z', 'xy', 'xz', 'yz', 'xyz']
norm = matplotlib.colors.BoundaryNorm(ticks, cmap.N)

# create figure, 3d grid, set background to white
fig2 = plt.figure(figsize=(12,8))
ax2 = fig2.add_subplot(111, projection='3d')
ax2.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot w/ median edit distance
img = ax2.scatter(c, k, median_dims, alpha=0.5, s=38, c=dim_num, cmap=cmap)

# plot horizontal plane for baseline
xx, yy = np.meshgrid(np.linspace(1,1.9), np.linspace(0,0.9))
#yy = np.meshgrid(range(2), range(2))
zz = xx * 0 + np.mean(rnn)
ax2.plot_surface(xx, yy, zz, alpha=0.5)

# set labels
ax2.set_ylabel('parameter k', fontsize=20, labelpad=10)
ax2.set_xlabel('parameter c', fontsize=20, labelpad=10)
ax2.set_zlabel('normalized edit distance', fontsize=20, labelpad=7)
#plt.title('Average edit distance (1 step)', fontsize=24)

# create colorbar
cb = plt.colorbar(img, cax = fig2.add_axes([0.82,0.3,0.03,0.4]))
cb.set_alpha(0.8)
cb.draw_all()
cb.ax.set_yticklabels(ticks, fontsize=16)
plt.show()

#plt.gca().xaxis.set_major_locator(plt.NullLocator())
#plt.gca().yaxis.set_major_locator(plt.NullLocator())
#plt.savefig('figures/test.svg', format='svg', bbox_inches='tight', pad_inches = 0)

/home/zauri/files/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: FixedFormatter should only be used together with FixedLocator


In [17]:
cpt = [4.0, 6.0, 6.0, 10.0, 7.0, 3.0, 5.0, 6.0, 6.0, 8.0, 7.0, 5.0, 11.0, 4.0, 4.0, 5.0, 9.0, 5.0, 11.0, 12.0, 9.0, 6.0, 12.0, 14.0, 7.0, 6.0, 9.0, 8.0, 6.0, 3.0, 5.0, 10.0, 6.0, 9.0, 6.0, 8.0, 8.0, 6.0, 8.0, 4.0, 3.0, 3.0, 7.0, 4.0, 9.0, 2.0, 3.0, 7.0, 5.0, 5.0, 4.0, 3.0, 13.0, 5.0, 6.0, 9.0, 7.0, 6.0, 7.0, 4.0, 3.0, 11.0, 1.0, 5.0, 5.0, 3.0, 10.0, 12.0, 6.0, 4.0, 6.0, 2.0, 9.0, 7.0, 6.0, 10.0, 8.0, 5.0, 8.0, 6.0, 4.0, 4.0, 4.0, 4.0, 2.0, 4.0, 3.0, 2.0, 2.0, 2.0, 4.0, 5.0, 3.0, 5.0, 5.0, 8.0, 4.0, 1.0, 4.0, 4.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 3.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]

In [18]:
rnn = [2.0, 6.0, 6.0, 9.0, 6.0, 2.0, 2.0, 4.0, 7.0, 5.0, 7.0, 5.0, 11.0, 5.0, 3.0, 5.0, 7.0, 2.0, 10.0, 10.0, 7.0, 3.0, 11.0, 13.0, 5.0, 6.0, 6.0, 6.0, 4.0, 2.0, 5.0, 11.0, 6.0, 8.0, 3.0, 7.0, 9.0, 4.0, 7.0, 2.0, 5.0, 4.0, 6.0, 4.0, 6.0, 3.0, 3.0, 6.0, 3.0, 6.0, 2.0, 4.0, 12.0, 3.0, 3.0, 7.0, 6.0, 5.0, 6.0, 4.0, 1.0, 8.0, 1.0, 5.0, 4.0, 3.0, 7.0, 10.0, 4.0, 5.0, 5.0, 2.0, 9.0, 8.0, 4.0, 7.0, 9.0, 6.0, 4.0, 5.0, 2.0, 3.0, 2.0, 4.0, 2.0, 3.0, 3.0, 2.0, 2.0, 2.0, 4.0, 5.0, 3.0, 5.0, 5.0, 7.0, 3.0, 1.0, 3.0, 4.0, 4.0, 6.0, 6.0, 5.0, 4.0, 2.0, 6.0, 3.0, 5.0, 4.0, 6.0, 6.0, 1.0, 3.0, 3.0, 3.0, 6.0, 3.0, 5.0, 4.0, 5.0, 6.0, 4.0, 6.0, 6.0, 2.0, 6.0, 5.0, 6.0, 6.0, 5.0, 3.0, 6.0, 6.0, 6.0, 6.0, 4.0, 6.0, 5.0, 5.0, 3.0, 5.0, 5.0, 4.0, 5.0, 6.0, 4.0, 5.0, 5.0, 5.0, 5.0, 6.0, 2.0, 2.0, 3.0, 3.0, 3.0, 2.0, 5.0, 4.0, 1.0, 3.0, 2.0, 3.0, 3.0, 4.0, 6.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 2.0, 5.0, 3.0, 3.0, 3.0, 3.0, 3.0]

In [19]:
len(cpt), len(rnn)

(180, 180)

In [54]:
worse_than_rnn = results_sum[lowest_idx_sum[0]].loc[results_sum[lowest_idx_sum[0]] >= 5.0].shape[0]

worse_than_rnn / len(df)

0.34444444444444444

### Plot best model with baselines

In [51]:
#%matplotlib inline
%matplotlib qt

IDs = df['ID']

error = df['error']

seqs= results_sum['sequence'][:-2].values

res = results_sum[lowest_idx_sum[0]][:-2].values
median = [np.nanmedian(res)] * len(res)

x = [x for x in range (0,len(seqs))]

# plot CPT baseline
plt.scatter(x, cpt, marker='o', s=20, c='dodgerblue', alpha=0.3, 
            label=str('CPT baseline median: ') + str(round(np.median(cpt),3)))
plt.plot(x, [np.median(cpt)] * len(x), '-', c='dodgerblue', alpha=0.9, linewidth=2)
#plt.plot((x_original,x_original),(res_original,distances), '--', c='darkgreen', alpha=0.6)
plt.fill_between(x, cpt, alpha=0.3, color='dodgerblue')


# plot RNN baseline
plt.scatter(x, rnn, marker='o', s=20, c='limegreen', alpha=0.5, 
            label=str('RNN baseline median: ') + str(round(np.median(rnn),3)))
plt.plot(x, [np.median(rnn)] * len(x), '-', c='green', alpha=0.9, linewidth=2)
#plt.plot((x_original,x_original),(res_original,distances), '--', c='darkgreen', alpha=0.6)
plt.fill_between(x, rnn, alpha=0.3, color='limegreen')

# plot random baseline
#plt.scatter(x, error, c='red', s=20, marker='o', alpha=0.3, 
#            label=str('guessing baseline median: ') + str(round(np.nanmedian(results_new['error']),3)))
#plt.plot(x, [np.median(error)] * len(x), c='red', alpha=0.8)#, 
         #label=str('guessing baseline avg.: ' + str(round(np.mean(results_original['error']),2))))
#plt.fill_between(x, error, [1.0 * len(error)], color='red', alpha=0.2)

# plot scatter + lines for simulations
plt.scatter(x, res, marker='o', s=26, c='darkviolet', alpha=0.8, 
            label=str('model-generated median: ') + str(round(lowest_median_sum,3)))
#plt.plot(x, res, c='blue', alpha=0.6)
plt.plot(x, median, c='darkviolet', alpha=0.95, linewidth=2)


plt.xticks(x, labels=IDs, rotation=90, fontsize=5)
#plt.xticklabels(IDs, rotation=90, fontsize=6)

plt.ylabel('accumulated prediction error', fontsize=22)
plt.xlabel('sequence', fontsize=22)
#plt.ylim(0.0, 0.51)
#plt.title('best model (all), ' + str(lowest_idx_sum[0]), fontsize=24, pad=20)
plt.margins(0.01)

plt.legend(fontsize=20, framealpha=0.8, loc='upper right', markerscale=2.5)

#plt.savefig('plot_median_editdist_individualerrors_diff.png', bbox_inches='tight')
plt.show()

### Prep data for stat analysis

In [34]:
# Compare avg edit distance for xy and xyz
list_xy = []
list_xyz = []

for col in results_mean:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xy':
        list_xy.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xyz':
        list_xyz.append(results_mean.at['mean',col])
        
avg_xy = np.mean(list_xy)
med_xy = np.median(list_xy)
std_xy = np.std(list_xy)
print('Average xy: ' + str(avg_xy)  + ', stdev: ' + str(std_xy) + ', median: ' + str(med_xy))
        
avg_xyz = np.mean(list_xyz)
med_xyz = np.median(list_xyz)
std_xyz = np.std(list_xyz)
print('Average xyz: ' + str(avg_xyz) + ', stdev: ' + str(std_xyz) + ', median: ' + str(med_xyz))

Average xy: 0.7278772290809328, stdev: 0.04127963384902352, median: 0.7305555555555555
Average xyz: 0.7264128943758573, stdev: 0.043342437700569406, median: 0.7194444444444444


In [35]:
# Compare avg edit distances for x, y, z
list_x = []
list_y = []
list_z = []
list_xz = []
list_yz = []

for col in results_mean:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'x':
        list_x.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'y':
        list_y.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'z':
        list_z.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xz':
        list_xz.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'yz':
        list_yz.append(results_mean.at['mean',col])

avg_xz = np.mean(list_xz)
avg_yz = np.mean(list_yz)
        
avg_x = np.mean(list_x)
lowest_x = min(list_x)
print('Average x: ' + str(avg_x) + ', min: ' + str(lowest_x)) 
        
avg_y = np.mean(list_y)
lowest_y = min(list_y)
print('Average y: ' + str(avg_y) + ', min: ' + str(lowest_y)) 
        
avg_z = np.mean(list_z)
lowest_z = min(list_z)
print('Average z: ' + str(avg_z) + ', min: ' + str(lowest_z)) 

print('Average xz: ' + str(avg_xz)) 
print('Average yz: ' + str(avg_yz)) 

Average x: 0.8538477366255143, min: 0.8055555555555556
Average y: 0.8213820301783264, min: 0.7388888888888889
Average z: 0.8785836762688615, min: 0.7833333333333333
Average xz: 0.8227366255144034
Average yz: 0.8185699588477368


In [36]:
# Compare all error predictions for dimensions
stat, p = friedmanchisquare(list_x,list_y,list_z,list_xy,list_xz,list_yz,list_xyz)
print('Friedman: stat = %.3f, p = % 10.3E' % (stat, p))

Friedman: stat = 3928.572, p =  0.000E+00


In [37]:
# 2D vs 3D median
stat, p = wilcoxon(list_xy, list_xyz, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 129915.000, p = 0.05785


In [52]:
# model vs RNN
stat, p = wilcoxon(rnn, res, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 3340.000, p = 0.00003


In [53]:
# model vs CPT
stat, p = wilcoxon(cpt, res, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 112.000, p = 0.00000
